### Test for constrained model with HMC/NUTS sampling

In [19]:
import os
import numpy as np
import pandas as pd

from fbprophet import Prophet

In [2]:
df = pd.read_csv(os.path.join('fbprophet','tests' ,'cp_20200613.txt'))

In [3]:
df.columns

Index(['y', 'ds', 'store_count', 'home_rec', 'discount_money', 'discount_gift',
       'attend_restrict_redeem', 'is_global_redeem', 'is_redeem',
       'is_flashsale', 'is_plan_market', 'weighted_redeem_level',
       'global_redeem_benefit', 'restrict_redeem_benefit',
       'weighted_discount_level', 'auto_clear_rate', 'in_stock_rate'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   y                        396 non-null    float64
 1   ds                       396 non-null    object 
 2   store_count              396 non-null    int64  
 3   home_rec                 396 non-null    int64  
 4   discount_money           396 non-null    int64  
 5   discount_gift            396 non-null    int64  
 6   attend_restrict_redeem   396 non-null    int64  
 7   is_global_redeem         396 non-null    float64
 8   is_redeem                396 non-null    float64
 9   is_flashsale             396 non-null    int64  
 10  is_plan_market           396 non-null    int64  
 11  weighted_redeem_level    396 non-null    int64  
 12  global_redeem_benefit    396 non-null    float64
 13  restrict_redeem_benefit  396 non-null    float64
 14  weighted_discount_level  3

In [5]:
df.head()

,y,ds,store_count,home_rec,discount_money,discount_gift,attend_restrict_redeem,is_global_redeem,is_redeem,is_flashsale,is_plan_market,weighted_redeem_level,global_redeem_benefit,restrict_redeem_benefit,weighted_discount_level,auto_clear_rate,in_stock_rate
0,27925.0,2020-05-17,42,0,0,0,0,0.0,0.0,3,16,0,0.0,0.0,42.039286,0.0,0.854349
1,1738.0,2020-05-11,42,0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0.000000,0.0,0.916997
2,3156.0,2019-10-07,40,0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0.000000,0.0,0.864427
3,3204.0,2020-03-15,42,0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0.000000,0.0,0.698065
4,2354.0,2020-04-17,42,0,0,0,0,0.0,0.0,0,16,0,0.0,0.0,0.000000,0.0,0.854861


In [6]:
df['weighted_discount_level'] = df['weighted_discount_level'].clip(0)
additive_regressor = list(set(df.columns) - set(['ds', 'y']))

In [7]:
additive_regressor

['is_plan_market',
 'discount_money',
 'is_redeem',
 'home_rec',
 'weighted_redeem_level',
 'is_global_redeem',
 'store_count',
 'attend_restrict_redeem',
 'weighted_discount_level',
 'in_stock_rate',
 'auto_clear_rate',
 'restrict_redeem_benefit',
 'global_redeem_benefit',
 'is_flashsale',
 'discount_gift']

In [8]:
model = Prophet(daily_seasonality=False, 
                weekly_seasonality=True, 
                yearly_seasonality=True,
                uncertainty_samples=False,
                stan_backend='PYSTAN',
                mcmc_samples=50,
               )

for i in additive_regressor:
    if i !='in_stock_rate':
        model.add_regressor(i, mode='additive', standardize='auto', constraints=[0, 1e5])
    else:
        model.add_regressor(i, mode='additive', standardize='auto', constraints=[0, 1e5])

In [9]:
model.fit(df, **{'init_r': 0.01})

/home/alexander/Documents/py_projects/git/prophet/fbprophet/stan_model/contrib/prophet_normal_truncated.pkl
Building model: prophet_normal_truncated.pkl
Model dir: /home/alexander/Documents/py_projects/git/prophet/fbprophet/stan_model/contrib
Name: prophet_normal_truncated.pkl


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_2e615812b02877d4b6be58eb42aec10e NOW.


Stan backend: PYSTAN


### Making quick diagnostics from stansummary
More: 
* https://betanalpha.github.io/assets/case_studies/rstan_workflow.html

* https://arxiv.org/pdf/1903.08008.pdf

In [48]:
print(model.fitted.stansummary())

Inference for Stan model: anon_model_2e615812b02877d4b6be58eb42aec10e.
4 chains, each with iter=50; warmup=25; thin=1; 
post-warmup draws per chain=25, total post-warmup draws=100.

                          mean se_mean     sd    2.5%     25%     50%     75%   97.5%  n_eff   Rhat
k                        -0.37    0.02    0.2   -0.69   -0.51   -0.38   -0.21    0.02    114    1.0
m                         0.16  3.5e-3   0.04    0.09    0.13    0.16    0.18    0.21    107    1.0
delta[1]                7.3e-3  6.7e-3   0.08   -0.19   -0.03  5.3e-3    0.04    0.16    139   0.98
delta[2]                  0.02  5.7e-3   0.06   -0.13   -0.02    0.01    0.06    0.14    120    1.0
delta[3]                  0.01  6.8e-3   0.07   -0.14   -0.02  8.7e-3    0.04    0.16     94   1.02
delta[4]                  0.01  5.1e-3   0.08   -0.14   -0.02    0.01    0.05    0.17    221   0.98
delta[5]                  0.03  6.5e-3   0.07   -0.08   -0.02  8.4e-3    0.05     0.2    121   0.98
delta[6]          

In [61]:
def get_stansummary(model):
    """
    Make a bit better stan summary
    """
    df = model.train_component_cols.copy(deep=True)
    df.drop(columns='multiplicative_terms', inplace=True)
    real_regname_map = dict()
    for i in df.columns:
        number_of_beta = df.iloc[np.flatnonzero(df[i])].index.values[0]
        real_regname_map.update({'beta[{}]'.format(number_of_beta+1) : i})
    strings_array = model.fitted.stansummary().split(sep='\n')[4:-5]
    values_array = [string.split() for string in strings_array]
    values_array[0].insert(0, 'var')
    dict_stansummary = dict(zip(values_array[0], zip(*values_array[1:])))
    help_list = list(dict_stansummary['var'])
    rev_subs = {beta: name for beta, name in real_regname_map.items()}
    help_list = [rev_subs.get(item, item) for item in help_list]
    dict_stansummary['var'] = tuple(help_list)
    df_stan_results = pd.DataFrame.from_dict(dict_stansummary, orient='columns')
    df_stan_results['Rhat'] = df_stan_results['Rhat'].astype(float)
    return df_stan_results

In [62]:
df_stan_summary = get_stansummary(model)
df_stan_summary

,var,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
0,k,-0.37,0.02,0.2,-0.69,-0.51,-0.38,-0.21,0.02,114,1.00
1,m,0.16,3.5e-3,0.04,0.09,0.13,0.16,0.18,0.21,107,1.00
2,delta[1],7.3e-3,6.7e-3,0.08,-0.19,-0.03,5.3e-3,0.04,0.16,139,0.98
3,delta[2],0.02,5.7e-3,0.06,-0.13,-0.02,0.01,0.06,0.14,120,1.00
4,delta[3],0.01,6.8e-3,0.07,-0.14,-0.02,8.7e-3,0.04,0.16,94,1.02
...,...,...,...,...,...,...,...,...,...,...,...
106,restrict_redeem_benefit,0.34,0.06,0.25,4.5e-3,0.13,0.29,0.53,0.85,18,1.35
107,global_redeem_benefit,0.3,0.05,0.2,0.02,0.13,0.29,0.43,0.7,16,1.27
108,is_flashsale,0.06,3.5e-4,3.4e-3,0.06,0.06,0.06,0.07,0.07,94,0.98
109,discount_gift,0.6,0.07,0.28,0.03,0.38,0.65,0.85,0.98,14,1.37


In [63]:
# Regressors with high difference in beetween-chain and within-chain variance
df_stan_summary[df_stan_summary['Rhat']>1.1]

,var,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
29,beta_constrained[2],0.46,0.08,0.3,0.01,0.17,0.46,0.76,0.94,15,1.40
30,beta_constrained[3],0.45,0.07,0.25,0.02,0.26,0.42,0.67,0.91,13,1.25
32,beta_constrained[5],0.43,0.06,0.25,0.06,0.24,0.41,0.61,0.95,20,1.32
33,beta_constrained[6],0.52,0.1,0.28,0.08,0.31,0.45,0.8,0.98,9,1.44
35,beta_constrained[8],0.51,0.13,0.32,5.0e-3,0.24,0.54,0.84,0.95,6,1.56
36,beta_constrained[9],7.8e-4,1.4e-4,6.4e-4,7.6e-5,3.0e-4,6.3e-4,1.2e-3,2.3e-3,20,1.15
37,beta_constrained[10],4.8e-3,1.4e-3,4.2e-3,4.5e-5,1.3e-3,3.7e-3,7.3e-3,0.02,8,1.19
38,beta_constrained[11],0.52,0.08,0.3,0.02,0.23,0.54,0.75,0.99,16,1.41
39,beta_constrained[12],0.34,0.06,0.25,4.5e-3,0.13,0.29,0.53,0.85,18,1.35
40,beta_constrained[13],0.3,0.05,0.2,0.02,0.13,0.29,0.43,0.7,16,1.27


Lets add more samples for Markov chains

In [66]:
model2 = Prophet(daily_seasonality=False, 
                weekly_seasonality=True, 
                yearly_seasonality=True,
                uncertainty_samples=False,
                stan_backend='PYSTAN',
                mcmc_samples=300,
               )

for i in additive_regressor:
    if i !='in_stock_rate':
        model2.add_regressor(i, mode='additive', standardize='auto', constraints=[0, 1e5])
    else:
        model2.add_regressor(i, mode='additive', standardize='auto', constraints=[0, 1e5])

In [67]:
model2.fit(df, **{'init_r': 0.01})

/home/alexander/Documents/py_projects/git/prophet/fbprophet/stan_model/contrib/prophet_normal_truncated.pkl
Stan backend: PYSTAN


In [68]:
df_stan_summary2 = get_stansummary(model)
df_stan_summary2

,var,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
0,k,-0.36,0.01,0.21,-0.77,-0.5,-0.35,-0.23,0.04,262,1.01
1,m,0.15,2.2e-3,0.04,0.08,0.13,0.15,0.18,0.22,270,1.00
2,delta[1],10.0e-3,2.4e-3,0.06,-0.12,-0.02,3.0e-3,0.03,0.18,738,1.00
3,delta[2],0.02,3.3e-3,0.07,-0.12,-0.02,0.02,0.06,0.19,476,1.00
4,delta[3],0.02,2.6e-3,0.07,-0.09,-0.01,0.02,0.05,0.18,628,1.00
...,...,...,...,...,...,...,...,...,...,...,...
106,restrict_redeem_benefit,0.46,0.01,0.29,0.02,0.2,0.44,0.71,0.97,692,1.00
107,global_redeem_benefit,0.46,0.01,0.28,0.03,0.22,0.46,0.68,0.96,502,1.00
108,is_flashsale,0.06,1.3e-4,3.8e-3,0.06,0.06,0.06,0.07,0.07,873,1.00
109,discount_gift,0.47,9.4e-3,0.27,0.03,0.24,0.44,0.69,0.97,823,1.00


In [69]:
# Regressors with high difference in beetween-chain and within-chain variance
df_stan_summary2[df_stan_summary2['Rhat']>1.1]

,var,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat


It seems that model2 converges. So the model specification fits the process in the data.

In [75]:
# Check statistics over posterior distributions for 'home_rec', 'is_plan_market', 'is_flashsale', 'store_count'
df_stan_summary2[df_stan_summary2['var'].isin(['home_rec', 'is_plan_market', 'is_flashsale', 'store_count'])]

,var,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
95,is_plan_market,0.03,1.6e-4,4.2e-3,0.02,0.02,0.03,0.03,0.03,643,1.0
98,home_rec,8.4e-3,1.7e-4,3.8e-3,1.2e-3,5.7e-3,8.3e-3,0.01,0.02,483,1.0
101,store_count,9.8e-3,3.8e-4,8.2e-3,3.9e-4,3.9e-3,7.7e-3,0.01,0.03,471,1.0
108,is_flashsale,0.06,1.3e-4,3.8e-3,0.06,0.06,0.06,0.07,0.07,873,1.0


In [82]:
future = model2.make_future_dataframe(periods=0)
for reg in additive_regressor:
    future[reg] = df[reg]
df_forecast = model2.predict(future)

In [98]:
df_forecast.head()

,ds,trend,additive_terms,attend_restrict_redeem,auto_clear_rate,discount_gift,discount_money,extra_regressors_additive,global_redeem_benefit,home_rec,...,is_plan_market,is_redeem,restrict_redeem_benefit,store_count,weekly,weighted_discount_level,weighted_redeem_level,yearly,multiplicative_terms,yhat
0,2019-05-08,9259.670100,8944.586615,0.0,0.0,0.0,0.0,13286.170490,0.0,-154.964321,...,1844.624518,0.0,0.0,972.442372,709.718692,24.892794,0.0,-5051.302566,0.0,18204.256715
1,2019-05-09,9202.530146,-7201.317569,0.0,0.0,0.0,0.0,-1337.643185,0.0,-154.964321,...,-1077.991372,0.0,0.0,972.442372,-734.124538,-10.653067,0.0,-5129.549846,0.0,2001.212577
2,2019-05-10,9145.390191,-8699.994939,0.0,0.0,0.0,0.0,-2776.683743,0.0,-154.964321,...,-1077.991372,0.0,0.0,-355.444453,-754.818596,-10.653067,0.0,-5168.492600,0.0,445.395252
3,2019-05-11,9088.250237,-6572.784124,0.0,0.0,0.0,0.0,-1800.549217,0.0,-154.964321,...,-1077.991372,0.0,0.0,972.442372,396.354245,-10.653067,0.0,-5168.589153,0.0,2515.466113
4,2019-05-12,9031.110283,-2536.902926,0.0,0.0,0.0,0.0,1453.592834,0.0,-154.964321,...,1844.624518,0.0,0.0,972.442372,1140.547288,-10.653067,0.0,-5131.043048,0.0,6494.207357
